<a href="https://colab.research.google.com/github/Crap89/IT_ACADEMY/blob/main/Tarea_M5_T02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios de Train-Test con ScikitLearn a partir de un dataset con información sobre las viviendas de Houston.

# Ejercicio 1:

Parte el conjunto de datos adjunto en train y test. Estudia ambos conjuntos por separado, a nivel descriptivo.

También adjunto encontrarás una descripción de las diferentes variables del dataset.


In [ ]:
# Importamos las librerías que vamos a necesitar:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
# Importamos los datos con los que vamos a trabajar:
df_housing = pd.read_csv("housing data.csv", header=None)
df_housing.head()

In [ ]:
# Asignamos los nombres a las columnas:
df_housing.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df_housing.head()

# Descripción de los atributos.

Estudio del dataset.

In [ ]:
# dimensiones del dataset
df_housing.shape

In [ ]:
# tipos de las variables:
df_housing.dtypes

In [ ]:
# búsqueda de valores vacíos:
df_housing.isna().sum()

In [ ]:
# búsqueda de valores nulos:
df_housing.isnull().sum()

In [ ]:
df_housing.describe().round(2).T

# Análisis previo de las variables del dataset.

Si bien vamos a centrarnos en analizar el subdataset sobre el que vamos a construir nuestro modelo (conjunto de datos Train), ver profundidad cada variable, sus relaciones entre ellas y las relaciones con la variable objetivo, haremos un análisis previo de las variables del dataset completo. Con este paso previo queremos ver si alguna variable no tiene relación ninguna con nuestra variable resultado, o si tienen relaciones de correlación que no aporten nada al estudio.

Relación entre variables del dataset completo:

In [ ]:
sns.pairplot(df_housing);

# Correlación entre variables del dataset completo.

Analizamos la correlación entre variables para ver si hay algunas que estén muy correlacionasdas y pueda afectar a nuestro modelo. También queremos ver si alguna de las variables atributo no está nada correlacionada con nuestra variable objetivo y no aporta nada al estudio.

In [ ]:
correlation_matrix = df_housing.corr()
plt.figure(figsize=(15,15))
sns.heatmap(correlation_matrix, cbar=True, fmt='.1f', annot=True, cmap='Blues');

# La información que extraemos de esta primera aproximación al dataset completo, junto con el resumen estadístico previo, es la siguiente:



*   La tasa de criminalidad (CRIM) y la proporción de suelo residencial (ZN)   aportan pocos datos, la mayoría son cero. 
*   La variable índice de acceso a la autopistas (RAD) es numérica ordinal.
*   La variable extensión del las orillas del río (CHAS) es binaria.
*   Salvo la distribución del valor medio de las viviendas (MEDV) y el número de habitaciones por vivienda (RM), el resto de variables tienen una distribución fuertemente asimétrica (cuartiles desequilibrados y desviaciones estardart altas) o tienen distribución no normal, necesitarán ser preprocesadas.
*   La mayorías de los datos están muy dispersos.
*  Vemos algunas correlaciones claras, lineales o polinómicas, entre estas variables:
     ◘ MEDV con RM y LSTAT
     ◘ LSTAT con NOX, RM, AGE, DIS, MEDV
     ◘ DIS con NOX, AGE, LSTAT, MEDV
     ◘ AGE con NOX, DIS, LSTAT, MEDV
     ◘ RM con LSTAT Y MEDV
     ◘ NOX con RM, AGE, DIS, LSTAT, MEDV
     ◘ CRIM con AGE, DIS, LSTST, MEDV

• No hay ninguna variable fuertemente correlacionada con nuestra variable objetivo y que influya negativamente en el estudio.



# Definimos nuestra variable objetivo/target.

A partir de este dataset queremos construir un modelo que nos permita predecir el valor medio de las casas ocupadas por sus propietarios (MEDV) en función de las características de la zona, esta será por tanto, nuestra variable respuesta.

In [ ]:
y = df_housing['MEDV'].copy()
y.shape

In [ ]:
# Definimos el dataset que contiene los atributos de nuestro estudio:
X = df_housing.drop('MEDV', axis=1).copy()
X.shape

Preparamos nuestro dataset para poder contruir nuestro modelo. Hacemos dos conjuntos, Train para entrenar nuestro modelo, y Test, para testear su eficacia. Tienen una proporción de 67% y 33% respectivamente.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Estudio de la variable objetivo "Valor medio de las casas ocupadas por sus propietarios por cada 1000$".

# Estudio de la distribución.

Nuestra variable objetivo es numérica continua. Vemos qué distribución tiene originalmente y en cada conjunto Train y Test después de dividir nuestro dataset.

In [ ]:
sns.kdeplot(df_housing['MEDV'], label='Distribución original', color='blue', fill=True)
sns.kdeplot(y_train, label='Distribución conjunto Train', color='red')
sns.kdeplot(y_test, label='Distribución conjunto Test', color='orange')
plt.title("Distribución original y distribuciones Train-Test", fontsize = 14)
plt.legend();

Vemos que tanto el conjunto Train como el conjunto Test tienen una distribución similar al conjunto original, no hace falta aplicar la técnica Data Shuffling para repartir mejor los datos.

# A continuación queremos saber si tiene una distribución normal. Planteamos las hipótesis:

Hipótesis nula H0, nuestra muestra tiene una distribución normal.               
Hipótesis H1, nuestra muestra no tiene una distribución normal.

Para validar nuestra hipótesis utilizamos el Test de distribución gaussiana Shapiro-Wilk con un alpha del 5%.

In [ ]:
from scipy.stats import shapiro
def distribucion_normal(data):
    stat, p = shapiro(data)
    print('stat=%.3f, p-value=%.20f' % (stat, p))
    if p > 0.05:
        print('Aceptamos la hipótesis nula H0: Probablemente Gaussiana')
    else:
        print('Rechazamos la hipótesis nula H0, nos quedamos con la alternativa H1: Probablemente no Gaussiana')

In [ ]:
distribucion_normal(y)

In [ ]:
# Estudio de la simetría, la forma y los outliers:
sns.boxplot(x = y)
plt.title("Análisis de Outliers", fontsize = 14);

In [ ]:
skewness = y.skew()
kurtosis = y.kurtosis()
print(f'Kurtosis= {kurtosis}', f'Skewness= {skewness}' )

In [ ]:
y.describe()

Ahora sabemos que nuestro target tiene una distribución no normal, asimétrica, con una asimetría positva (skewness con valores positivos). La curtosis positiva indica que tiene valores atípicos y que tiene una cola más alargada, la derecha. Es una variable con outliers y con poca dispersión.

Necesita un preprocesado con Robust Scaler

# Analizamos los atributos o variables independientes.

Para analizar cada atributo vamos a hacer una gráfica con sus distribuciones en el dataset original y después de dividir el dataset, tanto en el conjunto Train como en el conjunto Test. Para analizar los outliners hacemos un boxplot.

Hacemos un dataframe con los nombres de las columnas a estudiar para poder poner título a los gráficos con más comodidad.

In [ ]:
df_atributos = pd.DataFrame(
    {'columnas':['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'TAX', 'PTRATIO', 'B', 'LSTAT'],  
     'definiciones':['Tasa de criminalidad per capita por ciudad', 'Proporción de suelo residencial cada 25.000 pies cuadrados', 'Proporción de negocios mayoristas por acres', 'Concentración de óxido nitrítico (partes por 10 millones)', 'Media del número de habitaciones por vivienda', 'Proporción de viviendas contruidas antes de 1940 y habitadas por sus propietarios', 'Distancias ponderadas a cinco centros de empleos de Boston', 'Impuesto sobre el valor total de la propiedad por cada 10.000$', 'Ratio de alumnos por profesor', '1000(Bk - 0.63)^2 siendo Bk la proporción de ciudadanos negros por ciudad', 'Porcentaje de población bajo nivel adquisitivo']})
df_atributos.set_index('columnas', inplace=True)
df_atributos.head()

In [ ]:
 # hacemos una lista con los atributos que vamos a analizar:
atributos = list(df_atributos.index)  

In [ ]:
def analisis_atributo(colum):
    print('Análisis del atributo', df_atributos.definiciones[colum])
    print('')
    print('Análisis estadístico')
    # análisis estadístico
    print(X[colum].describe())
    print('')
    skewness = X[colum].skew()
    kurtosis = X[colum].kurtosis()
    print(f'Kurtosis= {kurtosis}')
    print(f'Skewness= {skewness}')
    print('')
    print('Test de normalidad')
    distribucion_normal(X_train[colum])
    print('')
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))
    axes = axes.flat    
    # análisis de la distibución
    sns.kdeplot(X[colum], label='Distribución original', color='gray', fill=True, ax = axes[0])
    sns.kdeplot(X_train[colum], label='Distribución conjunto Train', color='red', ax = axes[0])
    sns.kdeplot(X_test[colum], label='Distribución conjunto Test', color='blue', ax = axes[0])    
    axes[0].set_title("Distribución original (gris) y distribuciones Train (rojo) y Test (azul)", fontsize = 16)
    axes[0].set_xlabel(df_atributos.definiciones[colum], fontsize= 16) 
    axes[0].tick_params(labelsize = 10)
    print('')
    # análisis de los outliers
    sns.boxplot(x = X_train[colum])
    axes[1].set_title("Análisis de Outliers en el conjunto Train", fontsize = 16)
    axes[1].set_xlabel(df_atributos.definiciones[colum], fontsize = 16) 
    axes[1].tick_params(labelsize = 16)
    # análisis de la relación con la variable objetivo
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
    sns.regplot(
        x           = y_train,
        y           = X_train[colum],
        color       = "black",
        marker      = 'o',
        scatter_kws = {"alpha":0.7},
        line_kws    = {"color":"r","alpha":0.7},
        ax          = axes
    )
    axes.set_title(f"Valor medio de las casas ocupadas por sus propietarios por cada 1000$ y {df_atributos.definiciones[colum]}", fontsize = 16)
    axes.tick_params(labelsize = 12)
    axes.set_xlabel("Valor medio de las casas ocupadas por sus propietarios por cada 1000$", fontsize = 16)

In [ ]:
colum = 'CRIM'
analisis_atributo(colum)

La Tasa de criminalidad tiene una distribución unimodal no normal con una gran asimetría positiva, valor alto de skewness, debido a que la mayoría de sus valores son cero. También vemos una curtosis alta porque tiene valores atípicos y outliers. El resto de valores están muy dispersos.
La relación con la variable objetivo es lineal, la mayor criminalidad se da en zonas donde el valor medio de las viviendas es menor y a partir de cierto valor (alrededor de 25.000$) es casi inexistente.

# Proponemos un preprocesado con Robust Scaler para controlar los outliers.

In [ ]:
colum = 'ZN'
analisis_atributo(colum)

La distribución de la variable Proporción de suelo residencial es unimodal no normal, asimétrica positiva, con outliers.

La mayoría de los datos son cero, como si no se hubiera valorado en la mayoría de registros, el resto de datos no tienen una relación con una pauta con nuestra variable. Queda excluida en este ejercicio de nuestro modelo.

In [ ]:
colum = 'INDUS'
analisis_atributo(colum)

La variable Proporción de negocios mayoristas tiene una distribución bimodal no gaussiana, con alta dispersión en sus valores, sin outliers.

# Proponemos una normalización con StandardScaler:

In [ ]:
colum = 'NOX'
analisis_atributo(colum)

La variable Concentración de óxido nitrítico tiene una distribución unimodal no gaussiana, con alta dispersión en sus valores, sin outliers.

# Proponemos una normalización con MinMaxScaler:

In [ ]:
colum = 'RM'
analisis_atributo(colum)

La Media del número de habitaciones por vivienda tiene una distribución unimodal no normal con una kurtosis acentuada que refleja valores atípicos. Estos valores se observan en la gráficas como outliers y como un grupo de viviendas con alto valor, independientemente del número de habitaciones, pueden indicar un mala toma de datos.

La relación con la variable objetivo es evidente y lineal, a mayor número de habitaciones el valor medio de las viviendas aumenta.

# Proponemos un preprocesado con Robust Scaler para tratar los valores atípicos.

In [ ]:
colum = 'AGE'
analisis_atributo(colum)

La distribución de la proporción de viviendas construidas antes de 1940 tiene una distribución bimodal no gaussiana con asimetría negativa y curtosis negativa, valores atípicos sin outliers y una alta dispersión.

# Proponemos una normalización con MinMaxScaler.

In [ ]:
colum = 'DIS'
analisis_atributo(colum)

La distribución de la variable Distancias ponderadas a cinco centros de empleo es unimodal no normal, asimétrica positiva, con algún outlier.

Observamos que tiene relación con nuestra variable objetivo, a menos distancia menos valor medio.

# Proponemos un preprocesado con Robust Scaler:

In [ ]:
colum = 'TAX'
analisis_atributo(colum)

Impuesto sobre el valor total de la propiedad tiene una distribución bimodal no gaussiana, con alta dispersión en sus valores, sin outliers.

# Proponemos una normalización con MinMax Scaler.

In [ ]:
colum = 'PTRATIO'
analisis_atributo(colum)

La variable Ratio de alumno por profesor no tiene relación con nuestra variable objetivo, la dejamos fuera en este ejercicio del estudio de nuestro modelo.

In [ ]:
colum = 'B'
analisis_atributo(colum)

La Proporción de ciudadanos negros tiene una distribución unimodal asimétrica con una gran grupo de los valores en torno a un mismo rango de entre 300 y 400. Tiene outliers.

# Proponemos un preprocesado con Robust Scaler.

In [ ]:
colum = 'LSTAT'
analisis_atributo(colum)

La distribucion de la variable Nivel adquisito es unimodal no normal, con outliers.
Tiene una fuerte relación lineal con el Valor medio de la vivienda.

Proponemos un preprocesados con Robust Scaler.

#  Ejercicio 2:

Aplica algún proceso de transformación (estandarizar los datos numéricos, crear columnas dummies, polinomios...).

In [ ]:
# Creamos un dataset para filtrar las columnas con las que no vamos a trabajar y hacer las transformaciones.
X_trainFS = X_train.drop(['ZN','PTRATIO'] , axis=1).copy()
X_testFS = X_test.drop(['ZN','PTRATIO'] , axis=1).copy()

# Preprocesado de nuestros datos, Feature Scaling:

Ya tenemos el estudio de todas nuestras variables, vamos a preprocesarlas. El preprocesado debe hacerse para ambos conjuntos con los que vamos a entrenar y validar nuestro modelo.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# Variable Índice de accesibilidad a las autopistas radiales:

Esta variable es numérica ordinal, sólo puede tomar un rango de valores. Aplicamos un proceso de dummificación.

In [ ]:
# valores únicos de RED
X_trainFS.RAD.unique()

In [ ]:
X_trainFS.RAD.nunique() 

In [ ]:
 # total de registros por valor
X_trainFS.RAD.value_counts()

# Conjunto Train

In [ ]:
# creamos las columnas dummies
dummy_RAD = pd.get_dummies(X_trainFS["RAD"], prefix="RAD")
# eliminamos la variable RAD
X_trainFS = X_trainFS.drop(["RAD"], axis = 1)
# Añadimos las columnas dummies creadas al dataset original.
X_trainFS = pd.concat([X_trainFS, dummy_RAD], axis = 1)
X_trainFS.head()

# Conjunto Test:

In [ ]:
# creamos las columnas dummies
dummy_RAD = pd.get_dummies(X_testFS["RAD"], prefix="RAD")
# eliminamos la variable RAD
X_testFS = X_testFS.drop(["RAD"], axis = 1)
# Añadimos las columnas dummies creadas al dataset original.
X_testFS = pd.concat([X_testFS, dummy_RAD], axis = 1)
X_testFS.head()

# Variable Extensión de las orilas del Río Charles.

Esta variable es una variable dummie, no necesita preprocesado.

Variables con distribución no normal sin outliers, preprocesado para normalizar las distribuciones con MinMaxScaler.

Dataset Train.

In [ ]:
variablesNorm = ['INDUS', 'AGE', 'TAX', 'NOX']
mms = MinMaxScaler(feature_range = (0, 1))
X_trainFS[variablesNorm] = mms.fit_transform(X_train[variablesNorm])

# Dataset Test:

In [ ]:
X_testFS[variablesNorm] = mms.transform(X_test[variablesNorm])

# Variables con valores atípicos y ourliers, preprocesado de Robust Scaler.

Dataset Train.

In [ ]:
variablesRbSc = ['CRIM', 'RM', 'DIS', 'LSTAT','B']
rs = RobustScaler()
X_trainFS[variablesRbSc] = rs.fit_transform(X_train[variablesRbSc])

In [ ]:
# Dataset Test:
X_testFS[variablesRbSc] = rs.transform(X_test[variablesRbSc])

# Ejercicio 3:

Resume las nuevas columnas generadas de forma estadística y gráfica

In [ ]:
datos_graf = [X_train['INDUS'],X_train['NOX'],X_train['CRIM'],X_train['RM'],X_train['DIS'],X_train['TAX'],X_train['B'],X_train['LSTAT']]
fig = plt.figure(1, figsize=(12, 4))
ax = fig.add_subplot(111)
bp = ax.boxplot(datos_graf)
ax.set_title("Variables antes del preprocesado", fontsize = 12);

In [ ]:
datos_grafFS = [X_trainFS['INDUS'],X_trainFS['NOX'],X_trainFS['CRIM'],X_trainFS['RM'],X_trainFS['DIS'],X_trainFS['TAX'],X_trainFS['B'],X_trainFS['LSTAT']]
fig = plt.figure(1, figsize=(12, 4))
ax = fig.add_subplot(111)
bp = ax.boxplot(datos_grafFS)
ax.set_title("Variables después del preprocesado", fontsize = 12);

Tras el preprocesado las distribuciones están centradas y los outliers preparados para poder trabajar.

Nuestro dataset queda preparado para poder definir un modelo y entrenarlo.

In [ ]:
X_trainFS.head()

In [ ]:
X_testFS.head()